In [100]:
import pandas as pd 
from dataclasses import dataclass
from typing import List
from datetime import datetime


In [101]:
df_nov=pd.read_excel(r"..\Data\LineC_SessionData-individual rats_SampleData.xlsx")
df_march=pd.read_excel(r"..\Data\Snifftime data_March15.2019_example_withIndividualSniffs.xlsx")
df_sept=pd.read_excel(r"..\Data\AutomatedCage_Sept2023-24_raw data.xlsx")

In [102]:
df_sept.columns

Index(['RAT_NAME', 'ID_SAMPLE', 'LEVEL_NAME', 'RUN', 'HOLE', 'ID_BL_DOTS',
       'ID_GXP_DOTS', 'ID_BL_APOPO', 'EXECUTED', 'HIT', 'REWARD', 'SniffTime',
       'ReadTotalSnifftime', 'SESSION_DATE', 'ID_EVALUATION_SESSION',
       'STATUS_BLINDPOS', 'SAMPLE_TYPE', 'tblRAT_SESSION_REMARKS', 'REUSED',
       'tblEVALUATION_SniffThreshold', 'tblRAT_SESSION_SniffThreshold',
       'CONFIGURATION_NAME', 'TEMPERATURE', 'STATUS_KNOWNPOS', 'START_TIME',
       'END_TIME', 'DATE_INCOMING', 'tblEVALUATION_SESSION_REMARKS', 'Trainer',
       'Documenter', 'Handler', 'DOTS_NAME', 'ID_CONFIGURATION'],
      dtype='object')

In [143]:
rename_dict = {
    "RAT_NAME": "Rat_Name",
    "ID_EVALUATION_SESSION": "Evaluation_Session_ID",
    "RUN": "Run",
    "HOLE": "Hole",
    "SniffTime": "Sniff_Time",
    "tblEVALUATION_SniffThreshold": "Threshold_Time",
    'tblEVALUATION.SniffThreshold':"Threshold_Time",
    "SniffThreshold":"Threshold_Time",
    "SESSION_DATE" : "Date", 
    "ID_SESSION": "Sample_ID",
    "ID_SAMPLE": "Sample_ID",
    "age":"Age"
    # "ID_BL_DOTS": "Lab_Test"
}

In [144]:
df_sept = df_sept.rename(columns=rename_dict)

In [145]:
def _positive_sample_rat(row): 
    if row['Sniff_Time']>=row['Threshold_Time']: 
        return 1 
    else: 
        return 0

def _positive_sample_lab(row): 
    if row['ID_BL_DOTS']==0:
        return pd.NA
    elif row['ID_BL_DOTS']>1: 
        return 1
    else: 
        return 0


def false_positive(rat, lab):
    """Return 1 if rat says positive but lab test is negative, else 0"""
    return 1 if rat == 1 and lab == 0 else 0

def false_negative(rat, lab):
    """Return 1 if rat says negative but lab test is positive, else 0"""
    return 1 if rat == 0 and lab == 1 else 0

def true_positive(rat, lab):
    """Return 1 if rat says positive and lab test is positive, else 0"""
    return 1 if rat == 1 and lab == 1 else 0

def true_negative(rat, lab):
    """Return 1 if rat says negative and lab test is negative, else 0"""
    return 1 if rat == 0 and lab == 0 else 0


In [146]:
df_sept['Result_Of_Rat']=df_sept.apply(lambda x: _positive_sample_rat(x),axis=1)
df_sept['Lab_Test']=df_sept.apply(lambda x: _positive_sample_lab(x),axis=1)
df_sept["FP"] = df_sept.apply(lambda row: false_positive(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["FN"] = df_sept.apply(lambda row: false_negative(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["TP"]  = df_sept.apply(lambda row: true_positive(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["TN"]  = df_sept.apply(lambda row: true_negative(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["Age"]=""
df_sept["Weight"]=""
df_sept["Gender"]=""

In [ ]:
# Select and rename columns, add empty columns, and calculate new columns for df_march
df_march_mod = df_march.rename(columns=rename_dict)

# Keep only the required columns (including those to be created)
# columns_needed = [
#     'Rat_Name',
#     'Evaluation_Session_ID',
#     'Gender',
#     'Age',
#     'Weight',
#     'Run',
#     'Hole',
#     'Sniff_Time',
#     'Threshold_Time',
#     'Result_Of_Rat',
#     'Lab_Test',
#     "Date"
# ]

# Add empty columns for Gender and Weight
df_march_mod['Gender'] = ''
df_march_mod['Weight'] = ''
df_march_mod['Sniff_Time'] = df_march_mod['Sniff_Time'].fillna(0)

# Calculate Result_Of_Rat: 1 if Sniff_Time > Threshold_Time, else 0
df_march_mod['Result_Of_Rat'] = (df_march_mod['Sniff_Time'] > df_march_mod['Threshold_Time']).astype(int)

# Calculate Lab_Test: 1 if ID_BL_DOTS > 1, else 0 if == 1
df_march_mod['Lab_Test'] = df_march['ID_BL_DOTS'].apply(lambda x: 1 if x > 1 else (0 if x == 1 else None))

# Reorder columns to match the requested order


# Display the first few rows to check


,Date,Evaluation_Session_ID,Rat_Name,Run,Hole,LEVEL_NAME,Sniff_Time,Threshold_Time,tblRAT_SESSION.SniffThreshold,HIT,...,Sniff 5,Sniff 6,Sniff 7,Sniff 8,Cumulative Sniff Time,Match?,Gender,Weight,Result_Of_Rat,Lab_Test
0,2019-03-15,14097,Daudi,A,1,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,0,True,,,0,0
1,2019-03-15,14097,Daudi,A,2,2+,3831,2500,2500,True,...,NaN,NaN,NaN,NaN,3831,True,,,1,1
2,2019-03-15,14097,Daudi,A,3,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,0,True,,,0,0
3,2019-03-15,14097,Daudi,A,4,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,0,True,,,0,0
4,2019-03-15,14097,Daudi,A,5,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,0,True,,,0,0


In [ ]:
# Create FP, FN, TP, TN columns based on Lab_Test (ground truth) and Result_Of_Rat (prediction)
df_march_mod['TP'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['TN'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)
df_march_mod['FP'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['FN'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)



,Date,Evaluation_Session_ID,Rat_Name,Run,Hole,LEVEL_NAME,Sniff_Time,Threshold_Time,tblRAT_SESSION.SniffThreshold,HIT,...,Cumulative Sniff Time,Match?,Gender,Weight,Result_Of_Rat,Lab_Test,TP,TN,FP,FN
0,2019-03-15,14097,Daudi,A,1,-,0,2500,2500,False,...,0,True,,,0,0,0,1,0,0
1,2019-03-15,14097,Daudi,A,2,2+,3831,2500,2500,True,...,3831,True,,,1,1,1,0,0,0
2,2019-03-15,14097,Daudi,A,3,-,0,2500,2500,False,...,0,True,,,0,0,0,1,0,0
3,2019-03-15,14097,Daudi,A,4,-,0,2500,2500,False,...,0,True,,,0,0,0,1,0,0
4,2019-03-15,14097,Daudi,A,5,-,0,2500,2500,False,...,0,True,,,0,0,0,1,0,0


In [149]:
df_march_mod.columns

Index(['Date', 'Evaluation_Session_ID', 'Rat_Name', 'Run', 'Hole',
       'LEVEL_NAME', 'Sniff_Time', 'Threshold_Time',
       'tblRAT_SESSION.SniffThreshold', 'HIT', 'STATUS_BLINDPOS',
       'STATUS_KNOWNPOS', 'REWARD', 'Sample_ID', 'ID_GXP_DOTS', 'ID_BL_DOTS',
       'ID_BL_APOPO', 'ID_BL_FM', 'ID_GXP_APOPO', 'START_TIME', 'END_TIME',
       'Duration(M)', 'Duration(S)', 'TEMPERATURE', 'BIRTH_DATE', 'Age',
       'Sniff 1', 'Sniff 2', 'Sniff 3', 'Sniff 4', 'Sniff 5', 'Sniff 6',
       'Sniff 7', 'Sniff 8', 'Cumulative Sniff Time', 'Match?', 'Gender',
       'Weight', 'Result_Of_Rat', 'Lab_Test', 'TP', 'TN', 'FP', 'FN'],
      dtype='object')

In [150]:
# Select and rename only the columns you want
df_nov_selected = df_nov.rename(columns=rename_dict).copy()

# Calculate Age
df_nov_selected['Age'] = ((
    pd.to_datetime(df_nov['SESSION_DATE']) - pd.to_datetime(df_nov['Birthdate'])
).dt.days / 365.25).round(2)

df_nov_selected['Run'] = ''
df_nov_selected['Hole'] = ''
# Reorder to match your desired structure
final_columns = [
    'Rat_Name', 'Evaluation_Session_ID', 'Gender', 'Age', 'Weight',
    'Run', 'Hole', 'Sniff_Time', 'Threshold_Time', 'ID_BL_DOTS',"Date"
]



df_nov_final = df_nov_selected.copy()

df_nov_final.loc[:, 'Lab_Test'] = df_nov_final['ID_BL_DOTS'].apply(lambda x: 1 if x == 1 else 0)
df_nov_final = df_nov_final.drop(columns=['ID_BL_DOTS'])

df_nov_final = df_nov_final.copy()
df_nov_final.loc[:, 'Result_Of_Rat'] = (
    df_nov_final['Sniff_Time'] >= df_nov_final['Threshold_Time']
).astype(int)

df_nov_final = df_nov_final.copy()

# FP: Rat says Positive (1), Lab says Negative (0)
df_nov_final['FP'] = ((df_nov_final['Result_Of_Rat'] == 1) & (df_nov_final['Lab_Test'] == 0)).astype(int)

# FN: Rat says Negative (0), Lab says Positive (1)
df_nov_final['FN'] = ((df_nov_final['Result_Of_Rat'] == 0) & (df_nov_final['Lab_Test'] == 1)).astype(int)

# TP: Rat says Positive (1), Lab says Positive (1)
df_nov_final['TP'] = ((df_nov_final['Result_Of_Rat'] == 1) & (df_nov_final['Lab_Test'] == 1)).astype(int)

# TN: Rat says Negative (0), Lab says Negative (0)
df_nov_final['TN'] = ((df_nov_final['Result_Of_Rat'] == 0) & (df_nov_final['Lab_Test'] == 0)).astype(int)

In [151]:
columns_needed = [
    'Rat_Name',
    'Evaluation_Session_ID',
    'Gender',
    'Age',
    'Weight',
    'Run',
    'Hole',
    'Sniff_Time',
    'Threshold_Time',
    'Result_Of_Rat',
    'Lab_Test',
    "Date", 
    "TP",
    "FP",
    "TN",
    "FN", 
    "Sample_ID"
]

In [152]:
final_df= pd.concat([pd.concat([df_sept[columns_needed],df_march_mod[columns_needed]]),df_nov_final])


In [153]:
final_df.to_excel(r"..\Data\Report_data.xlsx")